# Modelo base 1 – Árbol de Decisión
# Clasificación de riesgo crediticio (Credit Score)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#X = pd.read_csv("X_processed.csv")
#y = pd.read_csv("y_labels.csv")["Credit_Score"]
X = pd.read_pickle("X_processed.pkl")
y = pd.read_pickle("y_labels.pkl")["Credit_Score"]

X.shape, y.shape


((100000, 6700), (100000,))

Train/test split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


Entrenamiento del Árbol de Decisión

In [4]:
clf = DecisionTreeClassifier(
    criterion="gini",
    class_weight="balanced",
    random_state=42
)

clf.fit(X_train, y_train)


,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,'balanced'


Métricas (train y test)

In [5]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

print("Accuracy train:", accuracy_score(y_train, y_train_pred))
print("Accuracy test:", accuracy_score(y_test, y_test_pred))

print("\nMatriz de confusión (test):")
print(confusion_matrix(y_test, y_test_pred))

print("\nClassification report (test):")
print(classification_report(y_test, y_test_pred))


Accuracy train: 1.0
Accuracy test: 0.70685

Matriz de confusión (test):
[[2330  125 1111]
 [ 184 4098 1517]
 [1270 1656 7709]]

Classification report (test):
              precision    recall  f1-score   support

        Good       0.62      0.65      0.63      3566
        Poor       0.70      0.71      0.70      5799
    Standard       0.75      0.72      0.74     10635

    accuracy                           0.71     20000
   macro avg       0.69      0.69      0.69     20000
weighted avg       0.71      0.71      0.71     20000



### Resultados del Modelo base 1: Árbol de Decisión

Se entrenó un Árbol de Decisión (`DecisionTreeClassifier`) utilizando todas las variables procesadas (`X_processed`) y la etiqueta `Credit_Score` como variable objetivo. La partición de los datos se realizó mediante `train_test_split` con un 80 % de datos para entrenamiento y 20 % para prueba, utilizando estratificación por clase para mantener las proporciones de las categorías de crédito.

Se configuró el modelo con criterio de impureza Gini y `class_weight="balanced"` para mitigar el desbalance entre las clases (Standard, Poor, Good).

Los resultados obtenidos fueron:

- **Accuracy en entrenamiento:** 1.00  
- **Accuracy en prueba:** ≈ 0.71  

El accuracy perfecto en entrenamiento indica que el árbol se ajusta totalmente a los datos de entrenamiento (sobreajuste), mientras que en prueba alcanza alrededor de un 71 % de acierto, lo que es razonable como modelo base inicial.

En cuanto al desempeño por clase (conjunto de prueba):

- **Good:** F1 ≈ 0.63 (precision ≈ 0.62, recall ≈ 0.65)  
- **Poor:** F1 ≈ 0.70 (precision ≈ 0.70, recall ≈ 0.71)  
- **Standard:** F1 ≈ 0.74 (precision ≈ 0.75, recall ≈ 0.72)  

La clase Standard, que es mayoritaria en el dataset, es la que obtiene mejores métricas, pero las clases Good y Poor también alcanzan un rendimiento aceptable. La media macro de F1 ronda 0.69, lo que muestra un comportamiento relativamente equilibrado entre las clases.

Este modelo se utilizará como **punto de referencia (baseline)**. En etapas posteriores se buscará reducir el sobreajuste y mejorar el rendimiento en prueba mediante:
- ajuste de hiperparámetros del árbol (profundidad máxima, tamaño mínimo de hoja),
- uso de modelos más robustos (por ejemplo, Random Forest o Regresión Logística),
- y técnicas adicionales de preprocesamiento como la normalización de variables para modelos sensibles a la escala.


In [6]:
from sklearn.ensemble import RandomForestClassifier

# Modelo 2: Random Forest
rf = RandomForestClassifier(
    n_estimators=100,        # número de árboles
    max_depth=None,         # luego podemos limitarlo si quieres
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight="balanced",# para el desbalance de clases
    n_jobs=-1,              # usa todos los núcleos disponibles
    random_state=42
)

rf.fit(X_train, y_train)
rf


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predicciones RF
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf  = rf.predict(X_test)

print("Accuracy train (RF):", accuracy_score(y_train, y_train_pred_rf))
print("Accuracy test  (RF):", accuracy_score(y_test, y_test_pred_rf))

print("\nMatriz de confusión (RF, test):")
print(confusion_matrix(y_test, y_test_pred_rf))

print("\nClassification report (RF, test):")
print(classification_report(y_test, y_test_pred_rf))


Accuracy train (RF): 1.0
Accuracy test  (RF): 0.76895

Matriz de confusión (RF, test):
[[2402   12 1152]
 [ 167 4447 1185]
 [ 845 1260 8530]]

Classification report (RF, test):
              precision    recall  f1-score   support

        Good       0.70      0.67      0.69      3566
        Poor       0.78      0.77      0.77      5799
    Standard       0.78      0.80      0.79     10635

    accuracy                           0.77     20000
   macro avg       0.76      0.75      0.75     20000
weighted avg       0.77      0.77      0.77     20000



4️⃣ Tabla comparativa Modelo 1 vs Modelo 2

Ahora armamos la tabla con métricas macro (Macro = promedio simple entre clases).

In [8]:
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# Asegurarnos de tener las predicciones del árbol
# (si ya tienes y_test_pred de antes, puedes reutilizarlo;
# si no, lo recalculas)
try:
    y_test_pred_dt = y_test_pred
except NameError:
    y_test_pred_dt = clf.predict(X_test)

# Métricas Árbol de Decisión (modelo 1)
prec_dt, rec_dt, f1_dt, _ = precision_recall_fscore_support(
    y_test, y_test_pred_dt, average="macro"
)
acc_dt = accuracy_score(y_test, y_test_pred_dt)

# Métricas Random Forest (modelo 2)
prec_rf, rec_rf, f1_rf, _ = precision_recall_fscore_support(
    y_test, y_test_pred_rf, average="macro"
)
acc_rf = accuracy_score(y_test, y_test_pred_rf)

# Tabla comparativa
resultados = pd.DataFrame({
    "Modelo": ["DecisionTree", "RandomForest"],
    "Accuracy": [acc_dt, acc_rf],
    "Precision_macro": [prec_dt, prec_rf],
    "Recall_macro": [rec_dt, rec_rf],
    "F1_macro": [f1_dt, f1_rf]
})

resultados


,Modelo,Accuracy,Precision_macro,Recall_macro,F1_macro
0,DecisionTree,0.70685,0.686192,0.694979,0.690339
1,RandomForest,0.76895,0.755367,0.747503,0.751283


### Comparación de modelos supervisados

Se entrenaron dos modelos de clasificación multiclase para predecir el puntaje crediticio (`Credit_Score`):

- **Modelo 1:** Árbol de Decisión (`DecisionTreeClassifier`).
- **Modelo 2:** Bosque Aleatorio (`RandomForestClassifier`).

Ambos modelos se evaluaron sobre el mismo conjunto de prueba (20 % de los datos), utilizando las métricas Accuracy, Precision macro, Recall macro y F1 macro. Los resultados se resumen en la siguiente tabla:

*(aquí va la tabla `resultados`)*

En general, el modelo **Random Forest** obtiene un mejor desempeño global que el Árbol de Decisión, especialmente en la métrica F1 macro, que refleja mejor el equilibrio entre las tres clases (Good, Poor y Standard). Además, al promediar muchos árboles, el Random Forest reduce el sobreajuste observado en el Árbol de Decisión individual.

Por estos motivos, se selecciona el **Random Forest** como **modelo principal** para el resto del proyecto (análisis con diferentes particiones, PCA y experimentos adicionales).
